# Degenerate perturbation theory coefficients

Code to compute the coefficients of degenerate perturbation theory.

Evaluates the recursive equation of the paper of Soliverez

    C E Soliverez (1969) J. Phys. C: Solid State Phys. 2 2161; DOI 10.1088/0022-3719/2/12/301
    
The Hamiltonian is 
$$
H = H_0 + V
$$
We note $P$ the projector on the degenerate subspace of $H_0$ of energy $E_0$.
The effective Hamiltonian in the subspace $P$ is written in terms of $P,V$ and the resolvant operator
$$
    R = (1 - P)/(E_0 - H).
$$

**Warning on convention**
Note that we have a difference of sign in our notations compared to [C E Soliverez (1969) J. Phys. C: Solid State Phys. 2 2161].

We use $R = (1 - P)/(E0 - H)$ instead of the $K = (1 - P)/(H - E0) = -R$ of Soliverez.

As a result, we have $[m_0, ..., m_n]_{here} = (-1)^{m_0 + ... + m_n} [m_0, ..., m_n]_{Soliverez}$.

This enables to remove the alternated sign in the simple term $PV(RV)^{r-1}P$ appearing at every order $r$ in the effective Hamiltonian.

In [1]:
import sympy as sy
from sympy.physics.quantum import HermitianOperator

P = HermitianOperator("P")
R = HermitianOperator("R")
V = HermitianOperator("V")

def L(n):
    if n == 0:
        return P
    elif n == 1:
        return R*V*P
    else:
        result = R*V*L(n-1)
        for l in range(1, n):
            result = result - R*L(l)*V*L(n-l-1)
        return result
    
def N(n):
    result = 0
    for l in range(0, n+1):
        result += L(l).adjoint()*L(n-l)
    return result

def N_posSqrRoot(n):
    if n == 0:
        return P
    elif n == 1:
        return 0
    else:
        result = sy.Rational(1,2)*N(n)
        for l in range(1, n):
            result += -sy.Rational(1,2)*N_posSqrRoot(l)*N_posSqrRoot(n-l)
        return result

def N_negSqrRoot(n):
    if n == 0:
        return P
    else:
        result = 0
        for l in range(n):
            result -= N_negSqrRoot(l)*N_posSqrRoot(n-l)
        return result

def SimplifyP(expr, order=10):
    """
    Simplify P^2 = P and RP=PR=0 in the expression.
    order: number of type we replace P^2 by P. Such that the maximum power of P in the expression must be smaller than 2^order.
    """
    for _ in range(1, order):
        expr = expr.subs(P**2, P)
    expr = expr.subs(R*P, 0)
    expr = expr.subs(P*R, 0)
    return expr


def Heff(n):
    result = 0
    for l in range(n):
        for m in range(n-l):
            result += N_posSqrRoot(l)*V*L(m)*N_negSqrRoot(n-l-m-1)
    return SimplifyP(result.expand())

def W(n):
    """
    Operator W of Soliverez associating the eigenstate of the total Hamiltonian |A> from the eigenstate |A'> of the effective Hamiltonian: 
        If    Heff |A'> = Delta |A'>
        Then  H|A> = (E0 + Delta)|A>
        where |A> = W|A'>.
    """
    result = 0
    for l in range(0, n+1):
        result += L(l)*N_negSqrRoot(n-l)
    return SimplifyP(result.expand())

In [2]:
## Effective Hamiltonian at a given order
Heff(4)

P*V*P*V*P*V*R**3*V*P/2 - P*V*P*V*R*V*R**2*V*P/2 - P*V*P*V*R**2*V*R*V*P/2 - P*V*R*V*P*V*R**2*V*P/2 + P*V*R*V*R*V*R*V*P - P*V*R*V*R**2*V*P*V*P/2 - P*V*R**2*V*P*V*R*V*P/2 - P*V*R**2*V*R*V*P*V*P/2 + P*V*R**3*V*P*V*P*V*P/2

In [4]:
## W at a given order
W(4)

-P*V*P*V*P*V*R**4*V*P/2 + P*V*P*V*R*V*R**3*V*P/2 + P*V*P*V*R**2*V*R**2*V*P/2 + P*V*P*V*R**3*V*R*V*P/2 - P*V*P*V*R**4*V*P*V*P/2 + P*V*R*V*P*V*R**3*V*P/2 - P*V*R*V*R*V*R**2*V*P/2 - P*V*R*V*R**2*V*R*V*P/2 + P*V*R*V*R**3*V*P*V*P/2 + 3*P*V*R**2*V*P*V*R**2*V*P/8 - P*V*R**2*V*R*V*R*V*P/2 + P*V*R**2*V*R**2*V*P*V*P/2 + P*V*R**3*V*P*V*R*V*P/2 + P*V*R**3*V*R*V*P*V*P/2 - P*V*R**4*V*P*V*P*V*P/2 + R*V*P*V*P*V*R**3*V*P/2 - R*V*P*V*R*V*R**2*V*P/2 - R*V*P*V*R**2*V*R*V*P/2 + R*V*P*V*R**3*V*P*V*P/2 - R*V*R*V*P*V*R**2*V*P/2 + R*V*R*V*R*V*R*V*P - R*V*R*V*R**2*V*P*V*P - R*V*R**2*V*P*V*R*V*P - R*V*R**2*V*R*V*P*V*P + R*V*R**3*V*P*V*P*V*P + R**2*V*P*V*P*V*R**2*V*P/2 - R**2*V*P*V*R*V*R*V*P + R**2*V*P*V*R**2*V*P*V*P - R**2*V*R*V*P*V*R*V*P - R**2*V*R*V*R*V*P*V*P + R**2*V*R**2*V*P*V*P*V*P + R**3*V*P*V*P*V*R*V*P + R**3*V*P*V*R*V*P*V*P + R**3*V*R*V*P*V*P*V*P - R**4*V*P*V*P*V*P*V*P

In [3]:
## Extract powers and coeff
def Extract_powers_and_coeff(expr):
    lcoeff  = []
    lpowers = []
    for term in expr.as_terms()[0]:
        lcoeff.append( term[1][0][0] )
        stringOp = term[1][2]
        powers = []
        for i in range(0,len(stringOp),2):
            if stringOp[i] == P:
                powers.append(0)
            else:
                powers.append(stringOp[i].as_powers_dict()[R])
        lpowers.append( powers )
    return lpowers, lcoeff

## Check it for some operator
lpowers, lcoeff = Extract_powers_and_coeff(Heff(3))
for powers, coeff in zip(lpowers, lcoeff):
    print(powers, coeff)

[0, 1, 1, 0] 1.0
[0, 2, 0, 0] -0.5
[0, 0, 2, 0] -0.5


In [6]:
## Save Heff coeff in a txt file
def Save_Heff(n):
    """
    Save coefficients of order n of the effective Hamiltonian in a txt file.
    Save it in 'DPT_coefficients_order_{n}.txt'
    """
    lpowers, lcoeff = Extract_powers_and_coeff(Heff(n))
    ## Data gathered in a list of list [ [m2, ..., mk, coeff], ...] for sorting. Ignore the first and last exponents which are necesserally 0.
    data = [[*powers[1:-1], coeff] for powers, coeff in zip(lpowers, lcoeff)]

    with open(f'DPT_coefficients_order_{n}.txt', 'w') as f:
        f.write(f"# Coefficient of the effective Hamiltonian in degenerated perturbation theory at order r={n}\n")
        f.write("# Format: <list of exponents [m_{r-1}, ..., m_{1}]> <coeff>\n")
        for m_and_coeff in sorted(data):
            f.write('\t'.join(str(mj) for mj in m_and_coeff[:-1])+'\t'+str(m_and_coeff[-1])+'\n')

Save_Heff(5)

In [ ]:
## Save W coeff in a txt file
def Save_W(n):
    """
    Save coefficients of order n of the effective Hamiltonian in a txt file.
    Save it in 'DPT_coefficients_order_{n}.txt'
    """
    lpowers, lcoeff = Extract_powers_and_coeff(W(n))
    ## Data gathered in a list of list [ [m1, m2, ..., mk, coeff], ...] for sorting. Ignore the last exponents which is necesserally 0.
    data = [[*powers[:-1], coeff] for powers, coeff in zip(lpowers, lcoeff)]

    with open(f'DPT_unitaryW_coefficients_order_{n}.txt', 'w') as f:
        f.write(f"# Coefficient of the unitary W of degenerated perturbation theory at order r={n}\n")
        f.write("# Format: <list of exponents [m_r, ..., m_{1}]> <coeff>\n")
        for m_and_coeff in sorted(data):
            f.write('\t'.join(str(mj) for mj in m_and_coeff[:-1])+'\t'+str(m_and_coeff[-1])+'\n')


Save_W(6)